<a href="https://colab.research.google.com/github/Piyush20-11/Abstractive-Text-Summarization/blob/main/ats_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import pickle
import string
import unicodedata
from random import randint

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from wordcloud import STOPWORDS, WordCloud

from sklearn.model_selection import train_test_split

In [2]:
!pip install -q contractions==0.0.48

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.4 MB/s eta 0:00:00


In [3]:
from contractions import contractions_dict

for key, value in list(contractions_dict.items())[:10]:
    print(f'{key} == {value}')

I'm == I am
I'm'a == I am about to
I'm'o == I am going to
I've == I have
I'll == I will
I'll've == I will have
I'd == I would
I'd've == I would have
Whatcha == What are you
amn't == am not


In [5]:
import csv

with open("Datasets/news_summary_more.csv", "r") as f:
    reader = csv.reader(f)
    data = list(reader)

# df = pd.DataFrame(data)
# df2 = pd.DataFrame(data2)
df1 = pd.DataFrame(data[1:], columns=data[0])


In [6]:
with open("Datasets/news_summary.csv", "r", encoding="latin-1") as f:
    reader = csv.reader(f)
    data = list(reader)
df2 = pd.DataFrame(data[1:], columns=data[0])


In [7]:
df1.head(5)

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [8]:
df2.head(5)

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [9]:
df2_columns = df2.columns.tolist()
df2_columns.remove('headlines')
df2_columns.remove('text')
df2.drop(df2_columns, axis='columns', inplace=True)

df = pd.concat([df1, df2], axis='rows')
del df1, df2

# Shuffling the df
df = df.sample(frac=1).reset_index(drop=True)

print(f'Dataset size: {len(df)}')
df.sample(5)

Dataset size: 8401


,headlines,text
6968,Some of Facebook's issues can never be fully s...,"In an end-of-year post, Facebook CEO Mark Zuck..."
5620,Sabarimala shut for purification after entry o...,The Sabarimala Temple in Kerala has been shut ...
1604,"5,000 chickens die after poultry farm catches ...","As many as 5,000 chickens were burnt alive at ..."
7939,Pay â¹100 cr by Friday or will order MD's arr...,The National Green Tribunal (NGT) has directed...
6937,British PM unveils Brexit 'Plan B' after origi...,British Prime Minister Theresa May on Monday u...


## Data preparation

In [10]:
def expand_contractions(text, contraction_map=contractions_dict):
    # Using regex for getting all contracted words
    contractions_keys = '|'.join(contraction_map.keys())
    contractions_pattern = re.compile(f'({contractions_keys})', flags=re.DOTALL)

    def expand_match(contraction):
        # Getting entire matched sub-string
        match = contraction.group(0)
        expanded_contraction = contraction_map.get(match)
        if not expand_contractions:
            print(match)
            return match
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


expand_contractions("y'all can't expand contractions i'd think")

'you all can not expand contractions id think'

In [11]:
# Converting to lowercase
df.text = df.text.apply(str.lower)
df.headlines = df.headlines.apply(str.lower)

df.sample(5)

,headlines,text
3124,songs recreated today don't have good melody: ...,"singer shaan said he isn't ""thrilled"" with the..."
7971,identify 36 'eco-smart' stations in 2 weeks: n...,the national green tribunal (ngt) has asked th...
5172,priests shut puri jagannath temple after tussl...,odisha's jagannath temple was closed on friday...
1191,no indian will be left out of nrc: pm narendra...,"addressing a gathering in assam's silchar, pm ..."
2314,twitter reacts to blonde look of priyanka in o...,reacting to an old picture of actress priyanka...


In [12]:
df.headlines = df.headlines.apply(expand_contractions)
df.text = df.text.apply(expand_contractions)
df.sample(5)

,headlines,text
989,"released by pakistan, hamid ansari crosses wag...",indian national hamid ansari crossed the attar...
3419,"have known hirani for yrs, what if metoo claim...",speaking about the sexual harassment allegatio...
5125,"airbus calls brexit process a disgrace, threat...",european planthemaker airbus ceo thomas enders...
6289,"11 killed as bus skids off road, falls into go...",at least 11 people have been killed and severa...
2566,ashwins away record has been shaky: harbhasingh,"speaking about spinner ravichandran ashwin, sp..."


In [13]:
# Remove puncuation from word
def rm_punc_from_word(word):
    clean_alphabet_list = [
        alphabet for alphabet in word if alphabet not in string.punctuation
    ]
    return ''.join(clean_alphabet_list)

print(rm_punc_from_word('#cool!'))


# Remove puncuation from text
def rm_punc_from_text(text):
    clean_word_list = [rm_punc_from_word(word) for word in text]
    return ''.join(clean_word_list)

print(rm_punc_from_text("Frankly, my dear, I don't give a damn"))

cool
Frankly my dear I dont give a damn
